In [ ]:
### Import all extra functions ###

from  plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as numpy
import dash
from dash import dcc
from dash import html, ctx
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
import time
import Utils ### Custom function scripts, Thanks to Tom Irons of the Teale group (UoN School of Chemistry) for molecule generation code. 19/7/22 ###
from collections import OrderedDict
import multiprocessing
import pandas as pd
from IPython.display import display, HTML
import json
display(HTML("<style>.container { width:100% !important; }</style>"))


# Variables input section

In [ ]:
### Variables ###
work_dir = "/data/Taxadiene_Charge/" ### Folder with all relevant files. Needs to end in / ###
steps = ["SOBD/","VERTD/","SOBE/"]
server = "LOCAL" ### Set to login001.augusta.nottingham.ac.uk for HPC access, if "LOCAL", no server will be used. ###
qm_package = "QChem" ### Set to either QChem or GAUSSIAN ###
host_work_dir = "/home/pcyra2/Taxadiene_Spherical/SOBD/" ### Folder on server for calculations. Needs to end in / ###
start_coordinate = "H2O.xyz" ### Currently only supports .xyz files, although new file types can be added easily ###
spherical_datapoints = 50 ### Number of single point calculations required ###
radius = 30 ### Radius of sphere ###
coord_filename = "coords_formatted.txt" ### Coord file, with format X, Y, Z, Atomic number (hint: H = 1, C = 6, N = 7, O = 8), (only change if adding own file) ###
energies_filename = "Energy.xyzc" ### Spherical energy file, with format X, Y, Z, C (only change if you're adding your own file) ###
resolution=100 ### Resolution of sphere, increase to improve contours but decrease for speed, Suggest 100 ###

# Plotting Section

In [ ]:
def parallel_interp(step):
    run_dir = str(work_dir) + str(step)
    # print(step+"\n")
    MolData = numpy.genfromtxt(run_dir + coord_filename)
    (x_data, y_data, z_data, en_data) =  Utils.EnergyExtract(run_dir,energies_filename)
    (x_sphere, y_sphere, z_sphere) = Utils.SphereGen(0,0,0,1,resolution)
    (col_sphere, x_contour, y_contour, z_contour, val_contour) = Utils.GenColors(x_sphere, y_sphere, z_sphere, x_data, y_data, z_data, en_data, radius)
    # print(col_sphere)
    columns = ["x_data", "y_data", "z_data", "en_data", "x_sphere", "y_sphere", "z_sphere","col_sphere", "x_contour", "y_contour", "z_contour", "val_contour"]
    df = pd.DataFrame(data = [x_data, y_data, z_data,en_data,x_sphere,y_sphere,z_sphere,col_sphere,x_contour,y_contour,z_contour,val_contour],)
    df = df.transpose()
    return (x_data, y_data, z_data,en_data,x_sphere,y_sphere,z_sphere,col_sphere,x_contour,y_contour,z_contour,val_contour,MolData)
    # return df

if __name__ == '__main__':
    pool = multiprocessing.Pool(len(steps)+1)
    processes = []
    start_time = time.perf_counter()
    result = pool.map(parallel_interp,steps)
    pool.close()
    finish_time = time.perf_counter()
    print(f"Time taken to interpolate color data is {finish_time - start_time} seconds.")

one = result[0]
two = result[1]
three = result[2]



In [ ]:
# app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX])
# app.layout = html.Div([
#     html.H1(children="Spherical visualisation tool",style={'text-align' : 'center'},className="bg-primary text-white m-1 p-3 text-decoration-underline"),
#     html.Div([
#         html.H3("Sphere opacity",style={'text-align' : 'center'},className="bg-secondary m-2 text-black"),
#         dcc.Slider(id='alphval', min=0, max=1, step=0.05, value=0.8, className="bg-secondary m-2 text-black"),
#     ], className="border border-3 m-2"),
#     html.Div([
#         html.Div([
#             html.H3("Interpolation data",style={'text-align' : 'center'}),
#             dcc.RadioItems(id='interp',options=["None","Sphere","Contours"],value="Sphere",inline=True, labelStyle={'margin-right' : '4px'} , className="p-2"),
#         ],className="vstack gap-2 m-2"),
#         html.Div([
#             html.H3("Molecule texture style",style={'text-align' : 'center'}),
#             dcc.RadioItems(id='texture',options=["matte","shiny","orbs"],value="orbs",inline=True,labelStyle={'margin-right' : '4px'}, className="p-2"),
#         ], className="vstack gap-2 m-2"),
#         html.Div([
#             html.H3("Molecule draw style",style={'text-align' : 'center'}),
#             dcc.RadioItems(id='draw_type',options=["ball_and_stick","tubes","wireframe","spacefilling"],value="tubes",inline=True,labelStyle={'margin-right' : '4px'}, className="p-2"),
#         ],className="vstack gap-2 m-2"),
#     ],className="hstack gap-3 bg-secondary m-2 text-black border border-3"),
# #     html.Div([dcc.Graph(id='graph')]),
# #     html.Div([
#     dbc.Row([
#         dbc.Col([
#             html.H3("Camera Information", style = {'text-align':'center'}),
#             html.P(id='camera'),
#             dbc.Col(),
#             dbc.Col(),
#             dbc.Col(),
#         ]),
#         dbc.Col([
#         dbc.Row(html.H3(children="Controls", style= {'text-align':'center'}),),
#         dbc.Row(dcc.Graph(id="control", figure = go.Figure(data=go.Surface(x=one[4]*radius, y=one[5]*radius, z=one[6]*radius)))),
#         ]),
#         dbc.Col(),
#     ],className="gap-0 bg-secondary m-2 text-black border border-3"),
#     dbc.Row([
#         dbc.Col([
# #         html.Div([
#             dbc.Row(html.H3(children="Reactant",style={'text-align' : 'center'}),),
#             dbc.Row(dcc.Graph(id="graph1", figure= go.Figure(data=go.Scatter3d())),),
#         ]),
# #     ], className = "vstack gap-2"),
# #         html.Div([
#         dbc.Col([
#             dbc.Row(html.H3(children="Transition State",style={'text-align' : 'center'}),),
#             dbc.Row(dcc.Graph(id="graph2", figure= go.Figure(data=go.Scatter3d())),),
#         ]),
# #     ], className = "vstack gap-2"),
# #         html.Div([
#         dbc.Col([
#             dbc.Row(html.H3(children="Product",style={'text-align' : 'center'}),),
#             dbc.Row(dcc.Graph(id="graph3",figure= go.Figure(data=go.Scatter3d())),),
#         ]),
# #     ], className="vstack gap-2"),   
#     ],className="gap-0"),
# #     ],className="hstack gap-0 border border-3 m-2"),
#     html.H1(id='debug')
# ])

# # @app.callback(Output('graph','figure'),Input('alphval','value'),Input('interp','value'),Input('texture','value'),Input('draw_type','value'))
# # def update_graph(alphval,interp,texture,draw_type):
# # #     Fig = make_subplots(rows=1, cols=3, subplot_titles = ("Rectant","Transition State", "Product"), specs=[[{"type":"scene"},{"type":"scene"},{"type":"scene"}]], horizontal_spacing=0.1)
# #     Fig = go.Figure(layout = go.Layout(title="Visualisation",uirevision='camera')).set_subplots(rows=1, cols=3, subplot_titles = ("Rectant","Transition State", "Product"), specs=[[{"type":"scene"},{"type":"scene"},{"type":"scene"}]], horizontal_spacing=0.01, shared_xaxes = True, shared_yaxes=True)
# # #     Fig.update_layout(autosize=True,)# width=1000, height=1000)
# #     Molecule = Utils.DrawMolecule(one[12],texture,draw_type) ### plot1
# #     for Bond in Molecule['bond_list']: Fig.add_trace(Bond,row=1,col=1)
# #     for Atom in Molecule['atom_list']: Fig.add_trace(Atom,row=1,col=1)
# #     Sphere = go.Surface(x=one[4]*radius, y=one[5]*radius, z=one[6]*radius, customdata=one[7], opacity=alphval,surfacecolor=one[7],contours={"x":{"show":True},"y":{"show":True},"z":{"show":True}},colorscale='Turbo')
# #     Energies = go.Scatter3d(x=one[0],y=one[1],z=one[2],mode='markers',marker=dict(size=5,color=one[3],colorscale='Turbo',opacity=1))
# #     Contours = go.Scatter3d(x=one[8],y=one[9],z=one[10],mode='markers',marker=dict(size=3,color='black'),hovertext=one[11])
# #     Fig.add_trace(Energies,row=1,col=1)
# #     if interp == "Sphere" or "Contours": Fig.add_trace(Sphere,row=1,col=1)
# #     if interp == "Contours":  Fig.add_trace(Contours,row=1,col=1)
# #     Molecule = Utils.DrawMolecule(two[12],texture,draw_type) ### plot2
# #     for Bond in Molecule['bond_list']: Fig.add_trace(Bond,row=1,col=2)
# #     for Atom in Molecule['atom_list']: Fig.add_trace(Atom,row=1,col=2)
# #     Sphere = go.Surface(x=two[4]*radius, y=two[5]*radius, z=two[6]*radius, customdata=two[7], opacity=alphval,surfacecolor=two[7],contours={"x":{"show":True},"y":{"show":True},"z":{"show":True}},colorscale='Turbo')
# #     Energies = go.Scatter3d(x=two[0],y=two[1],z=two[2],mode='markers',marker=dict(size=5,color=two[3],colorscale='Turbo',opacity=1))
# #     Contours = go.Scatter3d(x=two[8],y=two[9],z=two[10],mode='markers',marker=dict(size=3,color='black'),hovertext=two[11])
# #     Fig.add_trace(Energies,row=1,col=2)
# #     if interp == "Sphere" or "Contours": Fig.add_trace(Sphere,row=1,col=2)
# #     if interp == "Contours":  Fig.add_trace(Contours,row=1,col=2)
# #     Molecule = Utils.DrawMolecule(three[12],texture,draw_type) ### plot3
# #     for Bond in Molecule['bond_list']: Fig.add_trace(Bond,row=1,col=3)
# #     for Atom in Molecule['atom_list']: Fig.add_trace(Atom,row=1,col=3)
# #     Sphere = go.Surface(x=three[4]*radius, y=three[5]*radius, z=three[6]*radius, customdata=three[7], opacity=alphval,surfacecolor=three[7],contours={"x":{"show":True},"y":{"show":True},"z":{"show":True}},colorscale='Turbo')
# #     Energies = go.Scatter3d(x=three[0],y=three[1],z=three[2],mode='markers',marker=dict(size=5,color=three[3],colorscale='Turbo',opacity=1))
# #     Contours = go.Scatter3d(x=three[8],y=three[9],z=three[10],mode='markers',marker=dict(size=3,color='black'),hovertext=three[11])
# #     Fig.add_trace(Energies,row=1,col=3)
# #     if interp == "Sphere" or "Contours": Fig.add_trace(Sphere,row=1,col=3)
# #     if interp == "Contours":  Fig.add_trace(Contours,row=1,col=3)
# #     Fig.update_layout(showlegend=False)
# #     return Fig

# # @app.callback(Output('control','figure'))
# # def controller():
# #     Fig=go.Figure()
# #     Sphere = go.Surface(x=one[4]*radius, y=one[5]*radius, z=one[6]*radius,)
# #     Fig.add_trace(Sphere)
# #     return Fig
# @app.callback(Output('camera','children'), Input('control','relayoutData'))
# def cam_info(cam):
#     if cam and 'scene.camera' in cam:
#         camera = cam['scene.camera']
#         eye = camera['eye']
#         x=eye['x']
#         y=eye['y']
#         z=eye['z']
#         pos="X: "+str(numpy.round(x,2)*100)+"\tY: "+str(numpy.round(y,2)*100)+"\tZ: "+str(numpy.round(z,2)*100)
#     else:
#         pos = "X: 125\tY: 125\tZ:125"
#     return pos


# @app.callback(Output('graph1','figure'),Output('debug','children'),Input('alphval','value'),Input('interp','value'),Input('texture','value'),Input('draw_type','value'), Input('control','relayoutData'))#, Input('graph2','relayoutData'), Input('graph3','relayoutData'))
# def update_graph1(alphval,interp,texture,draw_type,cam1):#,cam2):
#     Fig = go.Figure()
#     config = {'staticPlot': True}
#     Fig.update_layout(autosize=True,)# width=1000, height=1000)
#     Molecule = Utils.DrawMolecule(one[12],texture,draw_type)
#     for Bond in Molecule['bond_list']: Fig.add_trace(Bond)
#     for Atom in Molecule['atom_list']: Fig.add_trace(Atom)
# #     Fig.update_layout(Utils.GetLayout(None))
#     # MinRange = numpy.min(Molecule['geometry'])
#     # MaxRange = numpy.max(Molecule['geometry'])
#     # Fig.update_layout(Utils.GetRange(MinRange,MaxRange))
#     Sphere = go.Surface(x=one[4]*radius, y=one[5]*radius, z=one[6]*radius, customdata=one[7], opacity=alphval,surfacecolor=one[7],contours={"x":{"show":True},"y":{"show":True},"z":{"show":True}},colorscale='Turbo')
#     Energies = go.Scatter3d(x=one[0],y=one[1],z=one[2],mode='markers',marker=dict(size=5,color=one[3],colorscale='Turbo',opacity=1))
#     Contours = go.Scatter3d(x=one[8],y=one[9],z=one[10],mode='markers',marker=dict(size=3,color='black'),hovertext=one[11])
#     Fig.add_trace(Energies)
#     if interp == "Sphere" or "Contours": Fig.add_trace(Sphere)
#     if interp == "Contours":  Fig.add_trace(Contours)
#     if cam1 and 'scene.camera' in cam1:
#         camera = cam1['scene.camera']    
#         Fig = Fig.update_layout(scene_camera=camera)
# #     if cam2 and 'scene.camera' in cam2:
# #         camera = cam2['scene.camera']
# #         Fig = Fig.update_layout(scene_camera= camera)
#     return Fig, "Debug Section"

        
# @app.callback(Output('graph2','figure'),Input('alphval','value'),Input('interp','value'),Input('texture','value'),Input('draw_type','value'), Input('control','relayoutData'))#Input('graph1','relayoutData'), Input('graph3','relayoutData'))
# def update_graph2(alphval,interp,texture,draw_type,cam1):#,cam2):
#     Fig = go.Figure()
#     config = {'staticPlot': True}
#     Fig.update_layout(autosize=True,)# width=1000, height=1000)
#     Molecule = Utils.DrawMolecule(two[12],texture,draw_type)
#     for Bond in Molecule['bond_list']: Fig.add_trace(Bond)
#     for Atom in Molecule['atom_list']: Fig.add_trace(Atom)
# #     Fig.update_layout(Utils.GetLayout(None))
#     # MinRange = numpy.min(Molecule['geometry'])
#     # MaxRange = numpy.max(Molecule['geometry'])
#     # Fig.update_layout(Utils.GetRange(MinRange,MaxRange))
#     Sphere = go.Surface(x=two[4]*radius, y=two[5]*radius, z=two[6]*radius, customdata=two[7], opacity=alphval,surfacecolor=two[7],contours={"x":{"show":True},"y":{"show":True},"z":{"show":True}},colorscale='Turbo')
#     Energies = go.Scatter3d(x=two[0],y=two[1],z=two[2],mode='markers',marker=dict(size=5,color=two[3],colorscale='Turbo',opacity=1))
#     Contours = go.Scatter3d(x=two[8],y=two[9],z=two[10],mode='markers',marker=dict(size=3,color='black'),hovertext=two[11])
#     Fig.add_trace(Energies)
#     if interp == "Sphere" or "Contours": Fig.add_trace(Sphere)
#     if interp == "Contours":  Fig.add_trace(Contours)
#     if cam1 and 'scene.camera' in cam1:
#         camera = cam1['scene.camera']    
#         Fig = Fig.update_layout(scene_camera=camera)
# #     if cam2 and 'scene.camera' in cam2:
# #         camera = cam2['scene.camera']
# #         Fig = Fig.update_layout(scene_camera= camera)
#     return Fig

# @app.callback(Output('graph3','figure'),Input('alphval','value'),Input('interp','value'),Input('texture','value'),Input('draw_type','value'), Input('control','relayoutData'))#, Input('graph1','relayoutData'), Input('graph2','relayoutData'))
# def update_graph3(alphval,interp,texture,draw_type,cam1):#,cam2):
#     Fig = go.Figure()
#     config = {'staticPlot': True}
#     Fig.update_layout(autosize=True,)# width=1000, height=1000)
#     Molecule = Utils.DrawMolecule(three[12],texture,draw_type)
#     for Bond in Molecule['bond_list']: Fig.add_trace(Bond)
#     for Atom in Molecule['atom_list']: Fig.add_trace(Atom)
# #     Fig.update_layout(Utils.GetLayout(None))
#     # MinRange = numpy.min(Molecule['geometry'])
#     # MaxRange = numpy.max(Molecule['geometry'])
#     # Fig.update_layout(Utils.GetRange(MinRange,MaxRange))
#     Sphere = go.Surface(x=three[4]*radius, y=three[5]*radius, z=three[6]*radius, customdata=three[7], opacity=alphval,surfacecolor=three[7],contours={"x":{"show":True},"y":{"show":True},"z":{"show":True}},colorscale='Turbo')
#     Energies = go.Scatter3d(x=three[0],y=three[1],z=three[2],mode='markers',marker=dict(size=5,color=three[3],colorscale='Turbo',opacity=1))
#     Contours = go.Scatter3d(x=three[8],y=three[9],z=three[10],mode='markers',marker=dict(size=3,color='black'),hovertext=three[11])
#     Fig.add_trace(Energies)
#     if interp == "Sphere" or "Contours": Fig.add_trace(Sphere)
#     if interp == "Contours":  Fig.add_trace(Contours)
#     if cam1 and 'scene.camera' in cam1:
#         camera = cam1['scene.camera']    
#         Fig = Fig.update_layout(scene_camera=camera)
# #     if cam2 and 'scene.camera' in cam2:
# #         camera = cam2['scene.camera']
# #         Fig = Fig.update_layout(scene_camera= camera)
#     config = {'staticPlot': True}
#     return Fig


# app.run_server(debug=True, use_reloader=False, host='0.0.0.0')

In [ ]:
### TEST.

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX])
app.layout = html.Div([
    html.H1(children="Spherical visualisation tool",style={'text-align' : 'center'},className="bg-primary text-white m-1 p-3 text-decoration-underline"),
    html.Div([
        html.H3("Sphere opacity",style={'text-align' : 'center'},className="bg-secondary m-2 text-black"),
        dcc.Slider(id='alphval', min=0, max=1, step=0.05, value=0.8, className="bg-secondary m-2 text-black"),
    ], className="border border-3 m-2"),
    html.Div([
        html.Div([
            html.H3("Interpolation data",style={'text-align' : 'center'}),
            dcc.RadioItems(id='interp',options=["None","Sphere","Contours"],value="Sphere",inline=True, labelStyle={'margin-right' : '4px'} , className="p-2"),
        ],className="vstack gap-2 m-2"),
        html.Div([
            html.H3("Molecule texture style",style={'text-align' : 'center'}),
            dcc.RadioItems(id='texture',options=["matte","shiny","orbs"],value="orbs",inline=True,labelStyle={'margin-right' : '4px'}, className="p-2"),
        ], className="vstack gap-2 m-2"),
        html.Div([
            html.H3("Molecule draw style",style={'text-align' : 'center'}),
            dcc.RadioItems(id='draw_type',options=["ball_and_stick","tubes","wireframe","spacefilling"],value="tubes",inline=True,labelStyle={'margin-right' : '4px'}, className="p-2"),
        ],className="vstack gap-2 m-2"),
    ],className="hstack gap-3 bg-secondary m-2 text-black border border-3"),
#     html.Div([dcc.Graph(id='graph')]),
#     html.Div([
    dbc.Row([
        dbc.Col([
            html.H3("Camera Information", style = {'text-align':'center'}),
            html.P(id="camera"),
            dbc.Col(
                [html.H4("X Position:", style = {'text-align':'center'}),
                dcc.Input(id='x',type="number", className="w-auto"),]
            ),
            dbc.Col(
                [html.H4("Y Position:", style = {'text-align':'center'}),
                dcc.Input(id='y',type="number", style = {'text-align':'center'}, className="w-auto", size='sm'),]
            ),
            dbc.Col(
                [html.H4("Z Position:", style = {'text-align':'center'}),
                dcc.Input(id='z',type="number", className="w-auto"),]
            
            ),
        ]),
        dbc.Col([
        dbc.Row(html.H3(children="Controls", style= {'text-align':'center'}),),
        dbc.Row(dcc.Graph(id="control", figure = go.Figure(data=go.Surface(x=one[4]*radius, y=one[5]*radius, z=one[6]*radius)))),
        ]),
        dbc.Col(),
    ],className="gap-0 bg-secondary m-2 text-black border border-3"),
    dbc.Row([
        dbc.Col([
#         html.Div([
            dbc.Row(html.H3(children="Reactant",style={'text-align' : 'center'}),),
            dbc.Row(dcc.Graph(id="graph1", figure= go.Figure(data=go.Scatter3d())),),
        ]),
#     ], className = "vstack gap-2"),
#         html.Div([
        dbc.Col([
            dbc.Row(html.H3(children="Transition State",style={'text-align' : 'center'}),),
            dbc.Row(dcc.Graph(id="graph2", figure= go.Figure(data=go.Scatter3d())),),
        ]),
#     ], className = "vstack gap-2"),
#         html.Div([
        dbc.Col([
            dbc.Row(html.H3(children="Product",style={'text-align' : 'center'}),),
            dbc.Row(dcc.Graph(id="graph3",figure= go.Figure(data=go.Scatter3d())),),
        ]),
#     ], className="vstack gap-2"),   
    ],className="gap-0"),
#     ],className="hstack gap-0 border border-3 m-2"),
    html.H1(id='debug')
])

@app.callback([Output('x','value'),Output('y','value'),Output('z','value'),Output('control','figure')],[Input('x','value'),Input('y','value'),Input('z','value'),Input('control','relayoutData'),Input('control','figure')])
def control_setting(x,y,z,cam, fig):
    trigger_id = ctx.triggered_id if not None else ""
    if trigger_id == "control":
        if cam and 'scene.camera' in cam:
            camera = cam['scene.camera']
            eye = camera['eye']
            x=numpy.round(eye['x']*100,2)
            y=numpy.round(eye['y']*100,2)
            z=numpy.round(eye['z']*100,2)
            fig=go.Figure(fig)
            return(x,y,z,fig)
        else:
            raise PreventUpdate
    else:
        fig = go.Figure(fig)
        if cam and 'scene.camera' in cam:
            camera = cam['scene.camera']
            eye = camera['eye']
            eye['x'] = float(x)/100
            eye['y'] = float(y)/100
            eye['z'] = float(z)/100
            camera['eye'] = eye
            fig = fig.update_layout(scene_camera=camera)
            return(x,y,z,fig)
        else:
            raise PreventUpdate

@app.callback(Output('camera','children'), Input('control','relayoutData'))
def cam_info(cam):
    if cam and 'scene.camera' in cam:
        camera = cam['scene.camera']
        eye = camera['eye']
        x=eye['x']
        y=eye['y']
        z=eye['z']
        pos="X: "+str(numpy.round(x,2)*100)+"\tY: "+str(numpy.round(y,2)*100)+"\tZ: "+str(numpy.round(z,2)*100)
    else:
        pos = "X: 125\tY: 125\tZ:125"
    return pos


@app.callback(Output('graph1','figure'),Output('debug','children'),Input('alphval','value'),Input('interp','value'),Input('texture','value'),Input('draw_type','value'), Input('control','relayoutData'), Input('graph1','figure'))#, Input('graph2','relayoutData'), Input('graph3','relayoutData'))
def update_graph1(alphval,interp,texture,draw_type,cam1, Fig):#,cam2):
    trigger_id = ctx.triggered_id if not None else ""
    if trigger_id != "control":
#         Fig = go.Figure()
        Fig = go.Figure()
#     Fig = go.Figure()
        config = {'staticPlot': True}
        Fig.update_layout(autosize=True,)# width=1000, height=1000)
        Molecule = Utils.DrawMolecule(one[12],texture,draw_type)
        for Bond in Molecule['bond_list']: Fig.add_trace(Bond)
        for Atom in Molecule['atom_list']: Fig.add_trace(Atom)

        Sphere = go.Surface(x=one[4]*radius, y=one[5]*radius, z=one[6]*radius, customdata=one[7], opacity=alphval,surfacecolor=one[7],contours={"x":{"show":True},"y":{"show":True},"z":{"show":True}},colorscale='Turbo')
        Energies = go.Scatter3d(x=one[0],y=one[1],z=one[2],mode='markers',marker=dict(size=5,color=one[3],colorscale='Turbo',opacity=1))
        Contours = go.Scatter3d(x=one[8],y=one[9],z=one[10],mode='markers',marker=dict(size=3,color='black'),hovertext=one[11])
        Fig.add_trace(Energies)
        if interp == "Sphere" or "Contours": Fig.add_trace(Sphere)
        if interp == "Contours":  Fig.add_trace(Contours)
        if cam1 and 'scene.camera' in cam1:
            camera = cam1['scene.camera']    
            Fig = Fig.update_layout(scene_camera=camera)
        return Fig, trigger_id
    else:
        Fig = go.Figure(Fig)
        if cam1 and 'scene.camera' in cam1:
            camera = cam1['scene.camera']    
            Fig = Fig.update_layout(scene_camera=camera)
        return Fig, trigger_id
               

        
@app.callback(Output('graph2','figure'),Input('alphval','value'),Input('interp','value'),Input('texture','value'),Input('draw_type','value'), Input('control','relayoutData'),Input('graph2','figure'))#Input('graph1','relayoutData'), Input('graph3','relayoutData'))
def update_graph2(alphval,interp,texture,draw_type,cam1,Fig):#,cam2):
    trigger_id = ctx.triggered_id if not None else ""
    if trigger_id != "control":
        Fig = go.Figure()
        config = {'staticPlot': True}
        Fig.update_layout(autosize=True,)# width=1000, height=1000)
        Molecule = Utils.DrawMolecule(two[12],texture,draw_type)
        for Bond in Molecule['bond_list']: Fig.add_trace(Bond)
        for Atom in Molecule['atom_list']: Fig.add_trace(Atom)
        Sphere = go.Surface(x=two[4]*radius, y=two[5]*radius, z=two[6]*radius, customdata=two[7], opacity=alphval,surfacecolor=two[7],contours={"x":{"show":True},"y":{"show":True},"z":{"show":True}},colorscale='Turbo')
        Energies = go.Scatter3d(x=two[0],y=two[1],z=two[2],mode='markers',marker=dict(size=5,color=two[3],colorscale='Turbo',opacity=1))
        Contours = go.Scatter3d(x=two[8],y=two[9],z=two[10],mode='markers',marker=dict(size=3,color='black'),hovertext=two[11])
        Fig.add_trace(Energies)
        if interp == "Sphere" or "Contours": Fig.add_trace(Sphere)
        if interp == "Contours":  Fig.add_trace(Contours)
        if cam1 and 'scene.camera' in cam1:
            camera = cam1['scene.camera']    
            Fig = Fig.update_layout(scene_camera=camera)
    else:
        Fig = go.Figure(Fig)
        if cam1 and 'scene.camera' in cam1:
            camera = cam1['scene.camera']    
            Fig = Fig.update_layout(scene_camera=camera)
    return Fig

@app.callback(Output('graph3','figure'),Input('alphval','value'),Input('interp','value'),Input('texture','value'),Input('draw_type','value'), Input('control','relayoutData'), Input('graph3','figure'))#, Input('graph1','relayoutData'), Input('graph2','relayoutData'))
def update_graph3(alphval,interp,texture,draw_type,cam1,Fig):#,cam2):
    trigger_id = ctx.triggered_id if not None else ""
    if trigger_id != "control":
        Fig = go.Figure()
        config = {'staticPlot': True}
        Fig.update_layout(autosize=True,)# width=1000, height=1000)
        Molecule = Utils.DrawMolecule(three[12],texture,draw_type)
        for Bond in Molecule['bond_list']: Fig.add_trace(Bond)
        for Atom in Molecule['atom_list']: Fig.add_trace(Atom)
        Sphere = go.Surface(x=three[4]*radius, y=three[5]*radius, z=three[6]*radius, customdata=three[7], opacity=alphval,surfacecolor=three[7],contours={"x":{"show":True},"y":{"show":True},"z":{"show":True}},colorscale='Turbo')
        Energies = go.Scatter3d(x=three[0],y=three[1],z=three[2],mode='markers',marker=dict(size=5,color=three[3],colorscale='Turbo',opacity=1))
        Contours = go.Scatter3d(x=three[8],y=three[9],z=three[10],mode='markers',marker=dict(size=3,color='black'),hovertext=three[11])
        Fig.add_trace(Energies)
        if interp == "Sphere" or "Contours": Fig.add_trace(Sphere)
        if interp == "Contours":  Fig.add_trace(Contours)
        if cam1 and 'scene.camera' in cam1:
            camera = cam1['scene.camera']    
            Fig = Fig.update_layout(scene_camera=camera)
    else:
        Fig = go.Figure(Fig)
        if cam1 and 'scene.camera' in cam1:
            camera = cam1['scene.camera']    
            Fig = Fig.update_layout(scene_camera=camera)
    return Fig


app.run_server(debug=True, use_reloader=False, host='0.0.0.0', )

In [ ]:
### TEST.

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX])
app.layout = html.Div([
    html.H1(children="Spherical visualisation tool",style={'text-align' : 'center'},className="bg-primary text-white m-1 p-3 text-decoration-underline"),
    html.Div([
        html.H3("Sphere opacity",style={'text-align' : 'center'},className="bg-secondary m-2 text-black"),
        dcc.Slider(id='alphval', min=0, max=1, step=0.05, value=0.8, className="bg-secondary m-2 text-black"),
    ], className="border border-3 m-2"),
    html.Div([
        html.Div([
            html.H3("Interpolation data",style={'text-align' : 'center'}),
            dcc.RadioItems(id='interp',options=["None","Sphere","Contours"],value="Sphere",inline=True, labelStyle={'margin-right' : '4px'} , className="p-2"),
        ],className="vstack gap-2 m-2"),
        html.Div([
            html.H3("Molecule texture style",style={'text-align' : 'center'}),
            dcc.RadioItems(id='texture',options=["matte","shiny","orbs"],value="orbs",inline=True,labelStyle={'margin-right' : '4px'}, className="p-2"),
        ], className="vstack gap-2 m-2"),
        html.Div([
            html.H3("Molecule draw style",style={'text-align' : 'center'}),
            dcc.RadioItems(id='draw_type',options=["ball_and_stick","tubes","wireframe","spacefilling"],value="tubes",inline=True,labelStyle={'margin-right' : '4px'}, className="p-2"),
        ],className="vstack gap-2 m-2"),
    ],className="hstack gap-3 bg-secondary m-2 text-black border border-3"),
#     html.Div([dcc.Graph(id='graph')]),
#     html.Div([
    dbc.Row([
        dbc.Col([
            html.H3("Camera Information", style = {'text-align':'center'}),
#             html.P(id="camera"),
            dbc.Row([dbc.Col(
                [html.H4("X Position:", style = {'text-align':'center'}),
                dcc.Input(id='x1',type="number", className="w-auto"),]
            ),
            dbc.Col(
                [html.H4("Y Position:", style = {'text-align':'center'}),
                dcc.Input(id='y1',type="number", style = {'text-align':'center'}, className="w-auto", size='sm'),]
            ),
            dbc.Col(
                [html.H4("Z Position:", style = {'text-align':'center'}),
                dcc.Input(id='z1',type="number", className="w-auto"),]
            
            ),])
        ]),
        dbc.Col([
        ]),
        dbc.Col(),
    ],className="gap-0 bg-secondary m-2 text-black border border-3"),
    dbc.Row([
        dbc.Col([
#         html.Div([
            dbc.Row(html.H3(children="Reactant",style={'text-align' : 'center'}),),
            dbc.Row(dcc.Graph(id="graph1", figure= go.Figure(data=go.Scatter3d())),),
        ]),
#     ], className = "vstack gap-2"),
#         html.Div([
        dbc.Col([
            dbc.Row(html.H3(children="Transition State",style={'text-align' : 'center'}),),
            dbc.Row(dcc.Graph(id="graph2", figure= go.Figure(data=go.Scatter3d())),),
        ]),
#     ], className = "vstack gap-2"),
#         html.Div([
        dbc.Col([
            dbc.Row(html.H3(children="Product",style={'text-align' : 'center'}),),
            dbc.Row(dcc.Graph(id="graph3",figure= go.Figure(data=go.Scatter3d())),),
        ]),
#     ], className="vstack gap-2"),   
    ],className="gap-0"),
#     ],className="hstack gap-0 border border-3 m-2"),
    html.H1(id='debug')
])

@app.callback([Output('x','value'),Output('y','value'),Output('z','value'),Output('control','figure')],[Input('x','value'),Input('y','value'),Input('z','value'),Input('control','relayoutData'),Input('control','figure')])
def control_setting1(x,y,z,cam1,cam2,cam3, fig):
    trigger_id = ctx.triggered_id if not None else ""
    if trigger_id == "control":
        if cam and 'scene.camera' in cam:
            camera = cam['scene.camera']
            eye = camera['eye']
            x=numpy.round(eye['x']*100,2)
            y=numpy.round(eye['y']*100,2)
            z=numpy.round(eye['z']*100,2)
            fig=go.Figure(fig)
            return(x,y,z,fig)
        else:
            raise PreventUpdate
    else:
        fig = go.Figure(fig)
        if cam and 'scene.camera' in cam:
            camera = cam['scene.camera']
            eye = camera['eye']
            eye['x'] = float(x)/100
            eye['y'] = float(y)/100
            eye['z'] = float(z)/100
            camera['eye'] = eye
            fig = fig.update_layout(scene_camera=camera)
            return(x,y,z,fig)
        else:
            raise PreventUpdate



@app.callback([Output('graph1','figure'),
              Output('x1','value'),
              Output('y1','value'),
              Output('z1','value')],
              [Input('alphval','value'),
               Input('interp','value'),
               Input('texture','value'),
               Input('draw_type','value'), 
               Input('graph1','figure'),
               Input('graph1','relayoutData'),
               Input('graph2','relayoutData'),
               Input('graph3','relayoutData'),
               Input('sync1','n_clicks'),
               Input('sync2','n_clicks'),
               Input('sync3','n_clicks'),
              ]),
def update_graph1(alphval,interp,texture,draw_type,Fig, cam1, cam2, cam3, b1,b2,b3 ):#,cam2):
    trigger_id = ctx.triggered_id if not None else ""
    if trigger_id == "sync1":
        Fig = go.Figure(Fig)
        if cam and 'scene.camera' in cam1:
            camera = cam1['scene.camera']
            eye = camera['eye']
            x = eye['x']
            y = eye['y']
            z = eye['z']
            Fig = Fig.update_layout(scene_camera=camera)
            return (Fig, x, y, z)
        else:
            raise PreventUpdate
    elif trigger_id == "b1":
        if cam and 'scene.camera' in cam1:
            camera = cam1['scene.camera']
            eye = camera['eye']
            eye['x'] = float(x)/100
            eye['y'] = float(y)/100
            eye['z'] = float(z)/100
            camera['eye'] = eye
            Fig = Fig.update_layout(scene_camera=camera)
            return (Fig, x, y, z)
        else:
            raise PreventUpdate  
    elif trigger_id == "b2":
        if cam and 'scene.camera' in cam2:
            camera = cam2['scene.camera']
            eye = camera['eye']
            eye['x'] = float(x)/100
            eye['y'] = float(y)/100
            eye['z'] = float(z)/100
            camera['eye'] = eye
            Fig = Fig.update_layout(scene_camera=camera)
            return (Fig, x, y, z)
        else:
            raise PreventUpdate      
    elif trigger_id == "b3":
        if cam and 'scene.camera' in cam3:
            camera = cam3['scene.camera']
            eye = camera['eye']
            eye['x'] = float(x)/100
            eye['y'] = float(y)/100
            eye['z'] = float(z)/100
            camera['eye'] = eye
            Fig = Fig.update_layout(scene_camera=camera)
            return (Fig, x, y, z)
        else:
            raise PreventUpdate  
    else:
        Fig = go.Figure()
        Fig.update_layout(autosize=True,)# width=1000, height=1000)
        Molecule = Utils.DrawMolecule(one[12],texture,draw_type)
        for Bond in Molecule['bond_list']: Fig.add_trace(Bond)
        for Atom in Molecule['atom_list']: Fig.add_trace(Atom)
        Sphere = go.Surface(x=one[4]*radius, y=one[5]*radius, z=one[6]*radius, customdata=one[7], opacity=alphval,surfacecolor=one[7],contours={"x":{"show":True},"y":{"show":True},"z":{"show":True}},colorscale='Turbo')
        Energies = go.Scatter3d(x=one[0],y=one[1],z=one[2],mode='markers',marker=dict(size=5,color=one[3],colorscale='Turbo',opacity=1))
        Contours = go.Scatter3d(x=one[8],y=one[9],z=one[10],mode='markers',marker=dict(size=3,color='black'),hovertext=one[11])
        Fig.add_trace(Energies)
        if interp == "Sphere" or "Contours": Fig.add_trace(Sphere)
        if interp == "Contours":  Fig.add_trace(Contours)
        if cam1 and 'scene.camera' in cam1:
            camera = cam1['scene.camera']    
            Fig = Fig.update_layout(scene_camera=camera)
        return Fig

               

        



app.run_server(debug=True, use_reloader=False, host='0.0.0.0', )